<a href="https://colab.research.google.com/github/amyush/Audio-steganography/blob/main/Audio_steganography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setting up the environment**

In [2]:
# Cloning the git repository to read the files
!git clone https://github.com/amyush/Audio-steganography.git

Cloning into 'Audio-steganography'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [3]:
import numpy as np
import pandas as pd
import cv2
from google.colab.patches import cv2
import wave

# **Custom Functions**

In [20]:
# Function to convert a string to its binary representation
def stringToBinary(a):
  l=[]
  res = ''.join(format(ord(i), '08b') for i in a)
  return res

# Function to convert a binary number to its respective string
def binaryToString(a):
  return chr(binaryToInt(a))

# Function to convert a integer to its binary representation
def integerToBinary(a):
  return int(bin(a)[2:])

# Function to convert a binary to its integer value
def binaryToInt(binary): 
    int_val, i, n = 0, 0, 0
    while(binary != 0): 
        a = binary % 10
        int_val = int_val + a * pow(2, i) 
        binary = binary//10
        i += 1
    return int_val

# **Steganography**

## **Hiding the data**

In [21]:
# Function to encode the message within an audio file
def encodeMessage(filename, key, message):
    # Base path of the code
    audioPath = "Audio-steganography/"

    # Reading the audio file as read-only
    audioFile = wave.open(audioPath + filename, 'rb')
    
    # Extract audio frame as byte array
    audioBytes = bytearray(list(audioFile.readframes(audioFile.getnframes())))
    
    # Declaring a delimeter
    delimeter = "#@#@#@"
    length = len(audioBytes)

    # Converting key and message to its binary representation
    # Here, custom function is used for conversion.
    binKey = stringToBinary(key)
    binMsg = stringToBinary(message + delimeter)
    lenBinKey = len(binKey)
    lenBinMsg = len(binMsg)

    # Pointer to keep track of current key bit
    index = -1
    # Pointer to keep track of current message bit
    count = 0
    pixel = 0

    # New audio file where the message is encoded
    encodedAudioBytes = audioBytes

    # Hiding the message in the audio file
    for j in range(0, length):
        index = index + 1;

        # Looping over key bits using modulus function to accomodate any length of message
        if((int)(binKey[index%lenBinKey]) == 1):
            # If the count is less, that means not all message is yet encoded
            if(count < lenBinMsg):
                pixel = integerToBinary(audioBytes[j])   
                newPixel = ((int)(pixel/10)*10) + (int)(binMsg[count])
                encodedAudioBytes[j] = binaryToInt(newPixel);
                count = count + 1;
            # Else, all the message alongwith delimeter is encoded. Thus, breaking out of loop
            else:
                break;

    # Coverting the encoded audio file to bytes
    encodedAudioBytes = bytes(encodedAudioBytes)
    # Writing the audio file to the base path
    newaudio = wave.open(audioPath + "encodedAudio.wav", "wb")
    # Setting the params of the original file to the encoded file
    newaudio.setparams(audioFile.getparams())
    # Writing frames of original audio to the encoded audio file
    newaudio.writeframes(encodedAudioBytes)

## **Extracting the data**

In [18]:
# Function to extract the message from an audio file
def extractData(key):
    imgPath ="Audio-steganography/"
    audioFile = wave.open(imgPath + 'encodedAudio.wav', 'rb')
    frame_bytes = bytearray(list(audioFile.readframes(audioFile.getnframes())))
    length = len(frame_bytes)
    
    binKey = stringToBinary(key)
    lenBinKey = len(binKey)
    delimeter = "#@#@#@"
    index = -1
    lenPixel = 0
    exMsg = ""
    newPixel = 0
    for j in range(0, length):
        index += 1
        if((int)(binKey[index%lenBinKey]) == 1):
            pixel = integerToBinary(frame_bytes[j])
            newPixel = newPixel * 10 + pixel%10
            lenPixel += 1
            if(lenPixel == 8):
                exMsg += binaryToString(newPixel)
                lenPixel = 0
                newPixel = 0
                if("#@#@#@" in exMsg):
                    return exMsg[:-6]
              

# **Results**

In [16]:
key = 'apple'
message = 'awdoajiwd ijawodjajwiojd'
encodeImage('cover_audio.wav', key, message)

In [19]:
extractedMessage = extractData(key)

print('encodedMessage = {}'.format(message))

print('extractedMessage = {}'.format(extractedMessage))

encodedMessage = awdoajiwd ijawodjajwiojd
extractedMessage = awdoajiwd ijawodjajwiojd
